In [1]:
#Solve Karnaugh Maps using Q-map Solver Scraper.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
import time

In [4]:
truth_table = ["""1	1	0	0	0	0	0
1	1	1	1	0	0	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1""", """1	1	1	1	1	1	1
1	1	0	0	0	0	0
1	1	1	1	0	0	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0
1	1	1	1	1	1	1
1	1	1	1	1	1	1""","""1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	0	0
1	1	1	1	0	0	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0
1	1	1	1	1	1	1""","""1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	1	1	1	1	1
1	1	0	0	0	0	0
1	1	1	1	0	0	0
0	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
1	1	1	1	1	1	1
0	1	0	1	0	1	0
0	0	0	1	0	0	0
0	1	1	0	0	0	1
1	0	0	1	0	0	0
1	1	1	1	0	0	1
1	1	0	1	0	1	0
0	1	1	0	0	0	0"""]

In [5]:
def raw_one_data(truth_table):
    """Converts list() of truth table to individual set. Does for one table only."""
    truth_table = truth_table.replace("\t", ",")
#     if(truth_table.index("X") != -1):
    truth_table = truth_table.replace("X", "2")
    truth_table = np.array(truth_table.split("\n")) 
    new_truth_table = []
    for i in range(len(truth_table)):
        one_val = truth_table[i].split(",")
        one_row = []
        for app in one_val:
            one_row.append(int(app))
        new_truth_table.append(one_row)
    return np.array(new_truth_table)

In [6]:
def process_data(table):
    """
    Process Each truth table.
    Returns a list of processed truth tables.
    """
    processed_truth_table = []
    for one_table in table:
        processed_truth_table.append(raw_one_data(one_table))
    return np.array(processed_truth_table)

In [7]:
def data_launch(br):
    """
    launches the driver session's k-map solver webpage.
    """
    br.get("http://www.32x8.com/var4.html") 
    table = br.find_elements_by_tag_name("table")[-1]
    rows = table.find_element_by_tag_name("tbody")
    rows = rows.find_elements_by_tag_name("tr")[2:]
    return rows

In [8]:
import chromedriver_binary ##--> Change the Chrome Version Build to v.78 for executable path.
br = webdriver.Chrome(executable_path=r'C:/Users/qasim/Downloads/chromedriver_win32_v78/chromedriver.exe')
rows = data_launch(br)

In [9]:
def insert_data(elem, data):
    """
    Inserts data from the np.array() of k-maps into the table
    on chrome-driver session.
    Submits the page.
    Parameters:
    1. elem: The set of np.array() elements with input field.
    2. data: np.array() 1x16 array of data.
    """
    
    insert = elem[0:-1]
    for value, radio_web in zip(data, insert):
        row = radio_web.find_elements_by_tag_name("input")
        for i in range(len(row)):
            if(i == value):
                row[i].click()
    elem[-1].find_elements_by_tag_name("input")[-1].click()

In [10]:
def one_hex_input(elem, data):
    """
    Returns a list of K-map equations for each individual LED display.
    Parameters:
    1. elem: np.array() row object.
    2. data: 16x7 vector of k-map truth table data.
    Returns:
    1. List: np.array() object of shape => 7x1, set of equations.
    """
    num = 0
    equations = []
    dataframes = []
    new_truth_table = data.T
    for x in new_truth_table:
        insert_data(elem, x)
        sop_elem = br.find_element_by_css_selector("div#sop").find_elements_by_css_selector("div")[5].text 

        sop_elem = sop_elem.replace("A", "Y₃")
        sop_elem = sop_elem.replace("B", "Y₂")
        sop_elem = sop_elem.replace("C", "Y₁")
        sop_elem = sop_elem.replace("D", "Y₀")
        sop_elem = sop_elem.replace("y", "D[" + str(num) + "]")

        df = br.find_element_by_css_selector("div#sop").find_elements_by_css_selector("div")[2].text  
        grid = []
        df = df.split("\n")[2:]
        for spl in df:
            grid.append(spl.split(" ")[1:])
        equations.append(sop_elem)
        df = pd.DataFrame(grid, index=['00', '01', '11', '10'], columns=['00', '01', '11', '10'])
        dataframes.append(df)
        elem = data_launch(br)
        num += 1
    return np.array(equations), dataframes

In [11]:
def create_func(rows, new_truth_table):
    k_maps = []
    grids = []
    for tables in new_truth_table:
        km, gr = one_hex_input(rows, tables)
        k_maps.append(km)
        grids.append(gr)
        rows = data_launch(br)
    return np.array(k_maps), grids

In [12]:
new_truth_table = process_data(truth_table)
k_maps, grids = create_func(rows, new_truth_table)
time.sleep(1)

In [13]:
# LED_rows = data_launch(br)
# new_LED_truth_table = process_data(LED_truth_table)
# led_kmaps, led_grids = create_func(LED_rows, new_LED_truth_table)

In [14]:
grids[0][0]
# grids[0][1]

,00,01,11,10
00,1,1,1,0
01,1,1,0,0
11,0,1,1,1
10,0,1,1,1


In [15]:
grids[0][0], grids[0][0]

(   00 01 11 10
 00  1  1  1  0
 01  1  1  0  0
 11  0  1  1  1
 10  0  1  1  1,    00 01 11 10
 00  1  1  1  0
 01  1  1  0  0
 11  0  1  1  1
 10  0  1  1  1)

In [16]:
k_maps

array([["D[0] = Y₃'Y₁' + Y₂'Y₀ + Y₁'Y₀ + Y₃Y₁",
        "D[1] = Y₁'Y₀' + Y₂Y₁' + Y₂Y₀' + Y₃Y₁ + Y₃'Y₂'Y₀",
        "D[2] = Y₃Y₀' + Y₃Y₂ + Y₃'Y₂'Y₀ + Y₃'Y₁'Y₀",
        "D[3] = Y₀ + Y₁ + Y₃'Y₂", "D[4] = Y₂Y₁'Y₀ + Y₃Y₂Y₁",
        "D[5] = Y₃'Y₂Y₁' + Y₂Y₁'Y₀ + Y₂Y₁Y₀' + Y₃Y₁Y₀",
        "D[6] = Y₃Y₂Y₁ + Y₂Y₁'Y₀ + Y₃Y₂'Y₀' + Y₃'Y₂'Y₁Y₀"],
       ["D[0] = Y₃'Y₁' + Y₃'Y₀' + Y₂Y₀' + Y₃Y₁",
        "D[1] = Y₂ + Y₃'Y₁' + Y₃'Y₀' + Y₃Y₀",
        "D[2] = Y₃'Y₀' + Y₃Y₀ + Y₂Y₁Y₀'", "D[3] = Y₀' + Y₁ + Y₃'Y₂",
        "D[4] = Y₂Y₁Y₀' + Y₃Y₂Y₁ + Y₃'Y₂'Y₁'Y₀'",
        "D[5] = Y₂Y₁ + Y₃'Y₂Y₀ + Y₃Y₂Y₀' + Y₃'Y₂'Y₁'Y₀'",
        "D[6] = Y₃'Y₁'Y₀' + Y₂Y₁Y₀' + Y₃Y₂'Y₀ + Y₃Y₁Y₀"],
       ["D[0] = Y₃'Y₂' + Y₃'Y₀ + Y₃'Y₁ + Y₁Y₀ + Y₃Y₂Y₁'",
        "D[1] = Y₃'Y₂' + Y₃'Y₁ + Y₂Y₀ + Y₃Y₀'",
        "D[2] = Y₃'Y₀ + Y₃'Y₂'Y₁' + Y₂Y₁Y₀ + Y₃Y₁Y₀' + Y₃Y₂Y₀'",
        "D[3] = Y₁' + Y₀ + Y₃'Y₂", "D[4] = Y₃'Y₂'Y₁' + Y₂Y₁Y₀",
        "D[5] = Y₃'Y₂'Y₁' + Y₂'Y₁'Y₀' + Y₃'Y₂Y₁ + Y₃Y₂Y₀",
        "D[6] = Y₃'Y₂'Y₁' + Y₃'Y₁'Y₀ + 

In [17]:
# LED_truth_table.replace("X", "2")
str(LED_truth_table).replace("X", "2")

NameError: name 'LED_truth_table' is not defined

In [ ]:
k_maps

In [ ]:
led_kmaps

In [ ]:
#32 K_maps with their tables in less than 45 seconds.

In [ ]:
led_grids[0][0]

In [ ]:
print("HEX Display K-Maps. Number = " + str(len(k_maps)*len(k_maps[0])))
i = 0
for maps in k_maps:
    print("HEX Display Number: " + str(i))
    for mx in maps:
        print("\n",mx)
    print("--------------------------------------")
    i += 1

In [ ]:
print("LED Display K-Maps. Number = " + str(len(led_kmaps[0])))
for lmap in led_kmaps[0]:
    print("\n",lmap)

In [ ]:
grids[3][6]

In [ ]:
led_kmaps

In [ ]:
LE_E2_test = ["""0	0	0	1
0	0	1	0
0	0	1	1
0	1	0	0
0	1	0	1
0	1	1	0
0	1	1	1
1	0	0	0
1	0	0	1
1	0	1	0
1	0	1	1
1	1	0	0
1	1	0	1
1	1	1	0
1	1	1	1
0	0	0	0"""]

In [ ]:
LE_rows = data_launch(br)
new_LE_E2_test = process_data(LE_E2_test)
le2_kmaps, le2_grids = create_func(LE_rows, new_LE_E2_test)

In [ ]:
led_grids[0][3]